In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import statistics

/Users/jagunmolu/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


^ - starts with, $ ends with

In [6]:
import os

# folder path
dir_path = r'Datasets/'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
print(res)

['~$5. Total_firms_Historical_data.xlsx', '5. Total_firms_Historical_data.xlsx', '4. share-of-the-population-with-access-to-electricity.csv', '2. national-strategy-for-youth-employment.csv', '3. share-of-education-in-government-expenditure.csv', '6. Country Codes.csv', '1. unemployment-rate-men-vs-women.csv']


In [186]:
unemployment_rate = pd.read_csv("Datasets/1. unemployment-rate-men-vs-women.csv")
national_strategy = pd.read_csv("Datasets/2. national-strategy-for-youth-employment.csv")
education = pd.read_csv("Datasets/3. share-of-education-in-government-expenditure.csv")
electricity = pd.read_csv("Datasets/4. share-of-the-population-with-access-to-electricity.csv")
firms_and_businesses = pd.read_excel("Datasets/5. Total_firms_Historical_data.xlsx", sheet_name=0)
country_codes = pd.read_csv("Datasets/6. Country Codes.csv")

In [187]:
unemployment_rate.head()

,Entity,Code,Year,"Unemployment, female (% of female labor force) (modeled ILO estimate)","Unemployment, male (% of male labor force) (modeled ILO estimate)",Population (historical estimates),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN
2,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN
3,Afghanistan,AFG,1993,13.712,10.165,14003764.0,NaN
4,Afghanistan,AFG,1994,13.722,10.170,15455560.0,NaN


In [188]:
cols = list(unemployment_rate.columns)
print(cols)

['Entity', 'Code', 'Year', 'Unemployment, female (% of female labor force) (modeled ILO estimate)', 'Unemployment, male (% of male labor force) (modeled ILO estimate)', 'Population (historical estimates)', 'Continent']


In [189]:
unemployment_rate.rename(
    columns={
        'Entity': 'country',
        'Unemployment, female (% of female labor force) (modeled ILO estimate)': 'female_unemployment',
        'Unemployment, male (% of male labor force) (modeled ILO estimate)': 'male_unemployment',
        'Population (historical estimates)': 'population'
    }, inplace=True)
unemployment_rate.head()

,country,Code,Year,female_unemployment,male_unemployment,population,Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN
2,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN
3,Afghanistan,AFG,1993,13.712,10.165,14003764.0,NaN
4,Afghanistan,AFG,1994,13.722,10.170,15455560.0,NaN


In [190]:
unemployment_rate.isna().sum()

country                    0
Code                    3529
Year                       0
female_unemployment    52347
male_unemployment      52347
population               326
Continent              58293
dtype: int64

In [191]:
unemployment_rate = unemployment_rate.dropna(subset=["female_unemployment", "male_unemployment"], how="all")
# unemployment_rate = unemployment_rate.dropna()
unemployment_rate.shape

(6231, 7)

In [192]:
unemployment_rate = unemployment_rate.dropna(subset=["Code"])
unemployment_rate.shape

(5828, 7)

In [193]:
cleaned_data = unemployment_rate.copy()

In [194]:
list(country_codes.columns)

['name', 'alpha-2', 'alpha-3', 'country-code', 'region', 'sub-region']

In [195]:
cleaned_data = cleaned_data.merge(country_codes[["name", "alpha-3", "region", "sub-region"]], left_on="Code", right_on="alpha-3", how="left")
cleaned_data.head()

,country,Code,Year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,sub-region
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,Southern Asia
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,Southern Asia
2,Afghanistan,AFG,1993,13.712,10.165,14003764.0,NaN,Afghanistan,AFG,Asia,Southern Asia
3,Afghanistan,AFG,1994,13.722,10.170,15455560.0,NaN,Afghanistan,AFG,Asia,Southern Asia
4,Afghanistan,AFG,1995,14.235,10.610,16418911.0,NaN,Afghanistan,AFG,Asia,Southern Asia


In [196]:
cleaned_data.shape

(5828, 11)

In [197]:
cols = list(cleaned_data["Code"].unique())
check = {x: x in list(country_codes["alpha-3"].unique()) for x in cols}
chk = [x for x in cols if x not in list(country_codes["alpha-3"])]
# print(check)
print(chk)

['OWID_CIS', 'OWID_WRL']


In [198]:
national_strategy.head(2)

,Entity,Code,Year,"8.b.1 - Existence of a developed and operationalized national strategy for youth employment, as a distinct strategy or as part of a national employment strategy - SL_CPA_YEMP"
0,Afghanistan,AFG,2019,1
1,Afghanistan,AFG,2020,1


In [199]:
list(national_strategy.columns)

['Entity',
 'Code',
 'Year',
 '8.b.1 - Existence of a developed and operationalized national strategy for youth employment, as a distinct strategy or as part of a national employment strategy - SL_CPA_YEMP']

In [200]:
national_strategy.rename(columns={
    list(national_strategy.columns)[-1]: 'national_strategy'
                         }, inplace=True)
national_strategy.head(2)

,Entity,Code,Year,national_strategy
0,Afghanistan,AFG,2019,1
1,Afghanistan,AFG,2020,1


In [201]:
cleaned_data.rename(columns={"Year": "year", "Code": "code"}, inplace=True)
cleaned_data.head(2)

,country,code,year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,sub-region
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,Southern Asia
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,Southern Asia


In [202]:
cleaned_data = cleaned_data.merge(national_strategy, left_on=["code", "year"], right_on=["Code", "Year"], how="left")
cleaned_data.head()

,country,code,year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,sub-region,Entity,Code,Year,national_strategy
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1993,13.712,10.165,14003764.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1994,13.722,10.170,15455560.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1995,14.235,10.610,16418911.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN


In [203]:
cleaned_data.shape

(5828, 15)

In [204]:
education.rename(columns={"Government expenditure on education, total (% of government expenditure)": "expenditure"}, inplace=True)
education.head(0)

,Entity,Code,Year,expenditure


In [205]:
education.head()

,Entity,Code,Year,expenditure
0,Afghanistan,AFG,2005,15.08
1,Afghanistan,AFG,2006,12.88
2,Afghanistan,AFG,2007,12.36
3,Afghanistan,AFG,2008,16.65
4,Afghanistan,AFG,2009,17.31


In [206]:
cleaned_data.shape

(5828, 15)

In [207]:
conf = cleaned_data.groupby("country").count()["female_unemployment"]
conf = conf.to_dict()
conf

{'Afghanistan': 31,
 'Albania': 31,
 'Algeria': 31,
 'Angola': 31,
 'Argentina': 31,
 'Armenia': 31,
 'Australia': 31,
 'Austria': 31,
 'Azerbaijan': 31,
 'Bahamas': 31,
 'Bahrain': 31,
 'Bangladesh': 31,
 'Barbados': 31,
 'Belarus': 31,
 'Belgium': 31,
 'Belize': 31,
 'Benin': 31,
 'Bhutan': 31,
 'Bolivia': 31,
 'Bosnia and Herzegovina': 31,
 'Botswana': 31,
 'Brazil': 31,
 'Brunei': 31,
 'Bulgaria': 31,
 'Burkina Faso': 31,
 'Burundi': 31,
 'Cambodia': 31,
 'Cameroon': 31,
 'Canada': 31,
 'Cape Verde': 31,
 'Central African Republic': 31,
 'Chad': 31,
 'Channel Islands': 31,
 'Chile': 31,
 'China': 31,
 'Colombia': 31,
 'Comoros': 31,
 'Congo': 31,
 'Costa Rica': 31,
 "Cote d'Ivoire": 31,
 'Croatia': 31,
 'Cuba': 31,
 'Cyprus': 31,
 'Czechia': 31,
 'Democratic Republic of Congo': 31,
 'Denmark': 31,
 'Djibouti': 31,
 'Dominican Republic': 31,
 'East Timor': 31,
 'Ecuador': 31,
 'Egypt': 31,
 'El Salvador': 31,
 'Equatorial Guinea': 31,
 'Eritrea': 31,
 'Estonia': 31,
 'Eswatini': 31,

In [208]:
print(conf)

{'Afghanistan': 31, 'Albania': 31, 'Algeria': 31, 'Angola': 31, 'Argentina': 31, 'Armenia': 31, 'Australia': 31, 'Austria': 31, 'Azerbaijan': 31, 'Bahamas': 31, 'Bahrain': 31, 'Bangladesh': 31, 'Barbados': 31, 'Belarus': 31, 'Belgium': 31, 'Belize': 31, 'Benin': 31, 'Bhutan': 31, 'Bolivia': 31, 'Bosnia and Herzegovina': 31, 'Botswana': 31, 'Brazil': 31, 'Brunei': 31, 'Bulgaria': 31, 'Burkina Faso': 31, 'Burundi': 31, 'Cambodia': 31, 'Cameroon': 31, 'Canada': 31, 'Cape Verde': 31, 'Central African Republic': 31, 'Chad': 31, 'Channel Islands': 31, 'Chile': 31, 'China': 31, 'Colombia': 31, 'Comoros': 31, 'Congo': 31, 'Costa Rica': 31, "Cote d'Ivoire": 31, 'Croatia': 31, 'Cuba': 31, 'Cyprus': 31, 'Czechia': 31, 'Democratic Republic of Congo': 31, 'Denmark': 31, 'Djibouti': 31, 'Dominican Republic': 31, 'East Timor': 31, 'Ecuador': 31, 'Egypt': 31, 'El Salvador': 31, 'Equatorial Guinea': 31, 'Eritrea': 31, 'Estonia': 31, 'Eswatini': 31, 'Ethiopia': 31, 'Fiji': 31, 'Finland': 31, 'France': 3

In [209]:
cleaned_data = cleaned_data.merge(education, left_on=["code", "year"], right_on=["Code", "Year"], how="left")
cleaned_data.head()

,country,code,year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,sub-region,Entity_x,Code_x,Year_x,national_strategy,Entity_y,Code_y,Year_y,expenditure
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1993,13.712,10.165,14003764.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1994,13.722,10.170,15455560.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1995,14.235,10.610,16418911.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
cleaned_data.shape

(5828, 19)

In [211]:
electricity.head(2)

,Entity,Code,Year,Access to electricity (% of population)
0,Afghanistan,AFG,2000,1.613591
1,Afghanistan,AFG,2001,4.074574


In [212]:
electricity.rename(columns={"Access to electricity (% of population)": "electricity_access"}, inplace=True)
electricity.head(1)

,Entity,Code,Year,electricity_access
0,Afghanistan,AFG,2000,1.613591


In [213]:
cleaned_data = cleaned_data.merge(electricity, left_on=["code", "year"], right_on=["Code", "Year"], how="left")
cleaned_data.head(2)

,country,code,year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,...,Year_x,national_strategy,Entity_y,Code_y,Year_y,expenditure,Entity,Code,Year,electricity_access
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
cleaned_data.shape

(5828, 23)

In [215]:
cleaned_data.drop(columns=["Entity", "Code", "Year"], inplace=True)
cleaned_data.head(2)

,country,code,year,female_unemployment,male_unemployment,population,Continent,name,alpha-3,region,sub-region,Entity_x,Code_x,Year_x,national_strategy,Entity_y,Code_y,Year_y,expenditure,electricity_access
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,NaN,Afghanistan,AFG,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
print(list(cleaned_data.columns))

['country', 'code', 'year', 'female_unemployment', 'male_unemployment', 'population', 'Continent', 'name', 'alpha-3', 'region', 'sub-region', 'Entity_x', 'Code_x', 'Year_x', 'national_strategy', 'Entity_y', 'Code_y', 'Year_y', 'expenditure', 'electricity_access']


In [217]:
cleaned_data.drop(columns=["Continent", "name", "alpha-3", "Entity_x", "Code_x", "Year_x", "Entity_y", "Code_y", "Year_y"], inplace=True)
cleaned_data.head(2)

,country,code,year,female_unemployment,male_unemployment,population,region,sub-region,national_strategy,expenditure,electricity_access
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,Asia,Southern Asia,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,Asia,Southern Asia,NaN,NaN,NaN


In [218]:
education.head(2)

,Entity,Code,Year,expenditure
0,Afghanistan,AFG,2005,15.08
1,Afghanistan,AFG,2006,12.88


In [219]:
unemployment_rate["Code"].unique()

array(['AFG', 'ALB', 'DZA', 'AGO', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE',
       'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BTN',
       'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA', 'BDI', 'KHM',
       'CMR', 'CAN', 'CPV', 'CAF', 'TCD', 'OWID_CIS', 'CHL', 'CHN', 'COL',
       'COM', 'COG', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'COD',
       'DNK', 'DJI', 'DOM', 'TLS', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI',
       'EST', 'SWZ', 'ETH', 'FJI', 'FIN', 'FRA', 'PYF', 'GAB', 'GMB',
       'GEO', 'DEU', 'GHA', 'GRC', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY',
       'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ',
       'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KWT',
       'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX',
       'MAC', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MRT', 'MUS',
       'MEX', 'MDA', 'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NPL',
       'NLD', 'NCL', 'NZL', 'NIC', 'NER', 'NGA', 'PRK', 'MKD', 'NOR',
       'OMN', '

In [220]:
country_codes["alpha-3"].unique()

array(['AFG', 'ALA', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'AIA', 'ATA',
       'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR',
       'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL',
       'BES', 'BIH', 'BWA', 'BVT', 'BRA', 'IOT', 'BRN', 'BGR', 'BFA',
       'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CYM', 'CAF', 'TCD', 'CHL',
       'CHN', 'CXR', 'CCK', 'COL', 'COM', 'COG', 'COD', 'COK', 'CRI',
       'CIV', 'HRV', 'CUB', 'CUW', 'CYP', 'CZE', 'DNK', 'DJI', 'DMA',
       'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH',
       'FLK', 'FRO', 'FJI', 'FIN', 'FRA', 'GUF', 'PYF', 'ATF', 'GAB',
       'GMB', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP',
       'GUM', 'GTM', 'GGY', 'GIN', 'GNB', 'GUY', 'HTI', 'HMD', 'VAT',
       'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL',
       'IMN', 'ISR', 'ITA', 'JAM', 'JPN', 'JEY', 'JOR', 'KAZ', 'KEN',
       'KIR', 'PRK', 'KOR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO',
       'LBR', 'LBY',

In [221]:
country_codes.head()

,name,alpha-2,alpha-3,country-code,region,sub-region
0,Afghanistan,AF,AFG,4,Asia,Southern Asia
1,Åland Islands,AX,ALA,248,Europe,Northern Europe
2,Albania,AL,ALB,8,Europe,Southern Europe
3,Algeria,DZ,DZA,12,Africa,Northern Africa
4,American Samoa,AS,ASM,16,Oceania,Polynesia


In [224]:
national_strategy.head()

,Entity,Code,Year,national_strategy
0,Afghanistan,AFG,2019,1
1,Afghanistan,AFG,2020,1
2,Argentina,ARG,2019,0
3,Argentina,ARG,2020,0
4,Argentina,ARG,2021,3


In [225]:
education.head()

,Entity,Code,Year,expenditure
0,Afghanistan,AFG,2005,15.08
1,Afghanistan,AFG,2006,12.88
2,Afghanistan,AFG,2007,12.36
3,Afghanistan,AFG,2008,16.65
4,Afghanistan,AFG,2009,17.31


In [226]:
electricity.head()

,Entity,Code,Year,electricity_access
0,Afghanistan,AFG,2000,1.613591
1,Afghanistan,AFG,2001,4.074574
2,Afghanistan,AFG,2002,9.409158
3,Afghanistan,AFG,2003,14.738506
4,Afghanistan,AFG,2004,20.064968


In [227]:
firms_and_businesses["Economy"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain*', 'Bangladesh',
       'Belarus', 'Belgium', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil ',
       'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Chile', 'China***',
       'Colombia', 'Congo, Dem. Rep.', 'Cook Islands*', 'Costa Rica',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominica*',
       'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Greece', 'Grenada*', 'Hong Kong SAR, China*', 'Hungary',
       'Iceland', 'India', 'Iran, Islamic Rep.', 'Iraq', 'Ireland',
       'Isle of Man*', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kazakhstan', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Latvia',
       'Lesotho', 'Liechtenstein*', 'Lithuania', 'Luxembourg',
       'Madagascar', 'Malaysia', 'Maldives', 'Malta', 'Mauritani

In [228]:
firms_and_businesses.head()

,Economy,Adult population,Year,TOTAL Number of \nLimited Liability Companies,Total business density rate
0,Afghanistan,14039528.0,2009.0,4289.0,0.305495
1,Afghanistan,14444996.0,2010.0,9048.0,0.626376
2,Afghanistan,15041824.0,2011.0,12793.0,0.850495
3,Afghanistan,15743958.0,2012.0,16350.0,1.038494
4,Afghanistan,16519812.0,2013.0,19541.0,1.182883


In [229]:
country_codes.head()

,name,alpha-2,alpha-3,country-code,region,sub-region
0,Afghanistan,AF,AFG,4,Asia,Southern Asia
1,Åland Islands,AX,ALA,248,Europe,Northern Europe
2,Albania,AL,ALB,8,Europe,Southern Europe
3,Algeria,DZ,DZA,12,Africa,Northern Africa
4,American Samoa,AS,ASM,16,Oceania,Polynesia


In [230]:
firms_and_businesses["Economy"] = firms_and_businesses["Economy"].str.replace("*", "",regex=False)
firms_and_businesses["Economy"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Belarus', 'Belgium', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil ',
       'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Chile', 'China',
       'Colombia', 'Congo, Dem. Rep.', 'Cook Islands', 'Costa Rica',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominica',
       'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Greece', 'Grenada', 'Hong Kong SAR, China', 'Hungary', 'Iceland',
       'India', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Isle of Man',
       'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan',
       'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Latvia', 'Lesotho',
       'Liechtenstein', 'Lithuania', 'Luxembourg', 'Madagascar',
       'Malaysia', 'Maldives', 'Malta', 'Mauritania', 'Mauri

In [231]:
firms_and_businesses["Economy"] = firms_and_businesses["Economy"].str.replace(r".+China", "China",regex=True)
firms_and_businesses["Economy"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Belarus', 'Belgium', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil ',
       'Brunei Darussalam', 'Bulgaria', 'Cambodia', 'Chile', 'China',
       'Colombia', 'Congo, Dem. Rep.', 'Cook Islands', 'Costa Rica',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominica',
       'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Estonia',
       'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany',
       'Greece', 'Grenada', 'Hungary', 'Iceland', 'India',
       'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Isle of Man', 'Israel',
       'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Korea, Rep.',
       'Kosovo', 'Kyrgyz Republic', 'Latvia', 'Lesotho', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Madagascar', 'Malaysia', 'Maldives',
       'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Mongol

In [232]:
firms_and_businesses["extra"] = firms_and_businesses["TOTAL Number of \nLimited Liability Companies"]\
/ (firms_and_businesses["Adult population"] / 1000)


In [233]:
firms_and_businesses[firms_and_businesses["Economy"] == "China"]#[["Economy", "Year", "Adult population"]]

,Economy,Adult population,Year,TOTAL Number of \nLimited Liability Companies,Total business density rate,extra
252,China,26612900.0,2010.0,610758.0,22.949697,22.949697
253,China,26900700.0,2011.0,675444.0,25.108789,25.108789
254,China,27153400.0,2012.0,740358.0,27.265757,27.265757
255,China,27424043.0,2013.0,822362.0,29.986899,29.986899
256,China,27364500.0,2014.0,995620.0,36.383636,36.383636
257,China,27261400.0,2015.0,1185401.0,43.482763,43.482763
258,China,27013000.0,2016.0,1328804.0,49.191278,49.191278
259,China,26731000.0,2017.0,1514928.0,56.673076,56.673076
260,China,26364300.0,2018.0,1643153.0,62.324924,62.324924
261,China,25996976.0,2019.0,1674699.0,64.418992,64.418992


In [234]:
temp = firms_and_businesses.groupby(["Economy", "Year"]).sum()
temp.head()

Adult population  \
Economy     Year                       
Afghanistan 2009.0        14039528.0   
            2010.0        14444996.0   
            2011.0        15041824.0   
            2012.0        15743958.0   
            2013.0        16519812.0   

                    TOTAL Number of \nLimited Liability Companies  \
Economy     Year                                                    
Afghanistan 2009.0                                         4289.0   
            2010.0                                         9048.0   
            2011.0                                        12793.0   
            2012.0                                        16350.0   
            2013.0                                        19541.0   

                    Total business density rate     extra  
Economy     Year                                           
Afghanistan 2009.0                     0.305495  0.305495  
            2010.0                     0.626376  0.626376  
            2011.0                     0.850495  0.850495  
            2012.0                     1.038494  1.038494  
            2013.0                     1.182883  1.182883

In [235]:
temp.reset_index(drop=False, inplace=True)
temp.head()

,Economy,Year,Adult population,TOTAL Number of \nLimited Liability Companies,Total business density rate,extra
0,Afghanistan,2009.0,14039528.0,4289.0,0.305495,0.305495
1,Afghanistan,2010.0,14444996.0,9048.0,0.626376,0.626376
2,Afghanistan,2011.0,15041824.0,12793.0,0.850495,0.850495
3,Afghanistan,2012.0,15743958.0,16350.0,1.038494,1.038494
4,Afghanistan,2013.0,16519812.0,19541.0,1.182883,1.182883


In [236]:
temp[temp["Economy"] == "China"]#[["Economy", "Year", "Adult population"]]

,Economy,Year,Adult population,TOTAL Number of \nLimited Liability Companies,Total business density rate,extra
252,China,2006.0,21387504.0,1208684.0,154.659912,154.659912
253,China,2007.0,21586946.0,1251301.0,163.708226,163.708226
254,China,2008.0,21765161.0,1284945.0,171.389021,171.389021
255,China,2009.0,21913827.0,1347998.0,182.171359,182.171359
256,China,2010.0,48573741.0,2057094.0,221.556322,221.556322
257,China,2011.0,48896498.0,2224740.0,240.695798,240.695798
258,China,2012.0,49199841.0,2386430.0,258.120644,258.120644
259,China,2013.0,49009507.0,2601367.0,284.725995,284.725995
260,China,2014.0,49373073.0,2901120.0,311.398604,311.398604
261,China,2015.0,49996870.0,3125242.0,320.966495,320.966495


In [237]:
temp.shape

(1496, 6)

In [238]:
temp.head()

,Economy,Year,Adult population,TOTAL Number of \nLimited Liability Companies,Total business density rate,extra
0,Afghanistan,2009.0,14039528.0,4289.0,0.305495,0.305495
1,Afghanistan,2010.0,14444996.0,9048.0,0.626376,0.626376
2,Afghanistan,2011.0,15041824.0,12793.0,0.850495,0.850495
3,Afghanistan,2012.0,15743958.0,16350.0,1.038494,1.038494
4,Afghanistan,2013.0,16519812.0,19541.0,1.182883,1.182883


In [239]:
temp["extra1"] = temp["TOTAL Number of \nLimited Liability Companies"]\
/ (temp["Adult population"] / 1000)
temp.head()

,Economy,Year,Adult population,TOTAL Number of \nLimited Liability Companies,Total business density rate,extra,extra1
0,Afghanistan,2009.0,14039528.0,4289.0,0.305495,0.305495,0.305495
1,Afghanistan,2010.0,14444996.0,9048.0,0.626376,0.626376,0.626376
2,Afghanistan,2011.0,15041824.0,12793.0,0.850495,0.850495,0.850495
3,Afghanistan,2012.0,15743958.0,16350.0,1.038494,1.038494,1.038494
4,Afghanistan,2013.0,16519812.0,19541.0,1.182883,1.182883,1.182883


In [240]:
temp.rename(columns={"TOTAL Number of \nLimited Liability Companies": "llcs", "extra1": "density_rate"}, inplace=True)
temp.head()

,Economy,Year,Adult population,llcs,Total business density rate,extra,density_rate
0,Afghanistan,2009.0,14039528.0,4289.0,0.305495,0.305495,0.305495
1,Afghanistan,2010.0,14444996.0,9048.0,0.626376,0.626376,0.626376
2,Afghanistan,2011.0,15041824.0,12793.0,0.850495,0.850495,0.850495
3,Afghanistan,2012.0,15743958.0,16350.0,1.038494,1.038494,1.038494
4,Afghanistan,2013.0,16519812.0,19541.0,1.182883,1.182883,1.182883


In [241]:
cleaned_data = cleaned_data.merge(temp, left_on=["country", "year"], right_on=["Economy", "Year"], how="left")
cleaned_data.head()

,country,code,year,female_unemployment,male_unemployment,population,region,sub-region,national_strategy,expenditure,electricity_access,Economy,Year,Adult population,llcs,Total business density rate,extra,density_rate
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1993,13.712,10.165,14003764.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1994,13.722,10.170,15455560.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1995,14.235,10.610,16418911.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
cleaned_data.drop(columns=["extra", "Total business density rate", "Economy", "Year"], inplace=True)
cleaned_data.head()

,country,code,year,female_unemployment,male_unemployment,population,region,sub-region,national_strategy,expenditure,electricity_access,Adult population,llcs,density_rate
0,Afghanistan,AFG,1991,13.615,10.089,10745168.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,13.816,10.258,12057436.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1993,13.712,10.165,14003764.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1994,13.722,10.170,15455560.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1995,14.235,10.610,16418911.0,Asia,Southern Asia,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5828 entries, 0 to 5827
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   country              5828 non-null   object 
 1   code                 5828 non-null   object 
 2   year                 5828 non-null   int64  
 3   female_unemployment  5828 non-null   float64
 4   male_unemployment    5828 non-null   float64
 5   population           5797 non-null   float64
 6   region               5766 non-null   object 
 7   sub-region           5766 non-null   object 
 8   national_strategy    278 non-null    float64
 9   expenditure          3350 non-null   float64
 10  electricity_access   4991 non-null   float64
 11  Adult population     1320 non-null   float64
 12  llcs                 1320 non-null   float64
 13  density_rate         1320 non-null   float64
dtypes: float64(9), int64(1), object(4)
memory usage: 683.0+ KB


In [110]:
cleaned_data.to_csv("cleaned_10alytics_dataset.csv")

In [244]:
cleaned_data.columns

Index(['country', 'code', 'year', 'female_unemployment', 'male_unemployment',
       'population', 'region', 'sub-region', 'national_strategy',
       'expenditure', 'electricity_access', 'Adult population', 'llcs',
       'density_rate'],
      dtype='object')

In [245]:
cleaned = cleaned_data.rename(columns={"region": "continent", "Adult population": "adult_population", "sub-region": "sub_region"})
cleaned = cleaned[["country", "code", "year", "continent", "sub_region", 'female_unemployment', 'male_unemployment', 'population'\
                            , "national_strategy", "expenditure", "adult_population", "electricity_access", "llcs", "density_rate"]]
cleaned.head()

,country,code,year,continent,sub_region,female_unemployment,male_unemployment,population,national_strategy,expenditure,adult_population,electricity_access,llcs,density_rate
0,Afghanistan,AFG,1991,Asia,Southern Asia,13.615,10.089,10745168.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1992,Asia,Southern Asia,13.816,10.258,12057436.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1993,Asia,Southern Asia,13.712,10.165,14003764.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1994,Asia,Southern Asia,13.722,10.170,15455560.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1995,Asia,Southern Asia,14.235,10.610,16418911.0,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
cleaned.isna().sum()

country                   0
code                      0
year                      0
continent                62
sub_region               62
female_unemployment       0
male_unemployment         0
population               31
national_strategy      5550
expenditure            2478
adult_population       4508
electricity_access      837
llcs                   4508
density_rate           4508
dtype: int64

In [247]:
cleaned.shape

(5828, 14)

In [248]:
cleaned_data.shape

(5828, 14)

In [249]:
cleaned.to_csv("cleaned_10alytics_dataset.csv")